In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/SolutionforVanSale

/content/drive/MyDrive/SolutionforVanSale


In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


sklearn.metrics has a mean_squared_error function with a squared kwarg (defaults to True). Setting squared to False will return the RMSE.  
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_actual, y_predicted, squared=False)

In [4]:
import random
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(seed=19)

In [5]:
# Đọc các file CSV
calendar = pd.read_csv("data/calendar.csv")
sales = pd.read_csv("data/sell_prices.csv")
train_evaluation = pd.read_csv("data/sales_train_evaluation.csv")

In [6]:
def downtype(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,j in enumerate(types):
        if pd.api.types.is_integer_dtype(j):
            if(df[cols[i]].min() >= np.iinfo(np.int8).min and df[cols[i]].max() <= np.iinfo(np.int8).max):
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif(df[cols[i]].min() >= np.iinfo(np.int16).min and df[cols[i]].max() <= np.iinfo(np.int16).max):
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif(df[cols[i]].min() >= np.iinfo(np.int32).min and df[cols[i]].max() <= np.iinfo(np.int32).max):
                df[cols[i]] = df[cols[i]].astype(np.int32)
            elif(df[cols[i]].min() >= np.iinfo(np.int64).min and df[cols[i]].max() <= np.iinfo(np.int64).max):
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif pd.api.types.is_float_dtype(j):
            if(df[cols[i]].min() >= np.finfo(np.float16).min and df[cols[i]].max() <= np.finfo(np.float16).max):
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif(df[cols[i]].min() >= np.finfo(np.float32).min and df[cols[i]].max() <= np.finfo(np.float32).max):
                df[cols[i]] = df[cols[i]].astype(np.float32)
            elif(df[cols[i]].min() >= np.finfo(np.float64).min and df[cols[i]].max() <= np.finfo(np.float64).max):
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif pd.api.types.is_object_dtype(j):
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

In [7]:
sales.head()

,Unnamed: 0,store_id,item_id,wm_yr_wk,sell_price
0,0,CA_1,HOBBIES_1_001,11325,9.58
1,1,CA_1,HOBBIES_1_001,11326,9.58
2,2,CA_1,HOBBIES_1_001,11327,8.26
3,3,CA_1,HOBBIES_1_001,11328,8.26
4,4,CA_1,HOBBIES_1_001,11329,8.26


In [8]:
calendar.head()

,Unnamed: 0,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [9]:
train_evaluation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [10]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   store_id    object 
 2   item_id     object 
 3   wm_yr_wk    int64  
 4   sell_price  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 261.0+ MB


In [11]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    1969 non-null   int64 
 1   date          1969 non-null   object
 2   wm_yr_wk      1969 non-null   int64 
 3   weekday       1969 non-null   object
 4   wday          1969 non-null   int64 
 5   month         1969 non-null   int64 
 6   year          1969 non-null   int64 
 7   d             1969 non-null   object
 8   event_name_1  162 non-null    object
 9   event_type_1  162 non-null    object
 10  event_name_2  5 non-null      object
 11  event_type_2  5 non-null      object
 12  snap_CA       1969 non-null   int64 
 13  snap_TX       1969 non-null   int64 
 14  snap_WI       1969 non-null   int64 
dtypes: int64(8), object(7)
memory usage: 230.9+ KB


In [12]:
train_evaluation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [8]:
sales.drop(columns= ["Unnamed: 0"], inplace=True)
calendar.drop(columns= ["Unnamed: 0"], inplace=True)

In [ ]:
sales.isna().sum()

,0
store_id,0
item_id,0
wm_yr_wk,0
sell_price,0


In [ ]:
train_evaluation.isna().sum()

,0
id,0
item_id,0
dept_id,0
cat_id,0
store_id,0
...,...
d_1937,0
d_1938,0
d_1939,0
d_1940,0


In [ ]:
calendar.isna().sum()

,0
date,0
wm_yr_wk,0
weekday,0
wday,0
month,0
year,0
d,0
event_name_1,1807
event_type_1,1807
event_name_2,1964


In [9]:
def pre_calendar(df):
    #loại bỏ các cột không cần thiết
    df = df.drop(['weekday', 'date', 'event_type_1', 'event_type_2'], axis=1)
    #đổi tên các cột
    df = df.assign(d = df.d.str[2:].astype(int))

    to_ordinal = ["event_name_1", "event_name_2"]
    df[to_ordinal] = df[to_ordinal].fillna('1')
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")

    return df

calendar = pre_calendar(calendar)
calendar.head()

,wm_yr_wk,wday,month,year,d,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI
0,11101,1,1,2011,1,1,1,0,0,0
1,11101,2,1,2011,2,1,1,0,0,0
2,11101,3,1,2011,3,1,1,0,0,0
3,11101,4,2,2011,4,1,1,1,1,0
4,11101,5,2,2011,5,1,1,1,0,1


In [20]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   wm_yr_wk      1969 non-null   int64
 1   wday          1969 non-null   int8 
 2   month         1969 non-null   int8 
 3   year          1969 non-null   int64
 4   d             1969 non-null   int64
 5   event_name_1  1969 non-null   int8 
 6   event_name_2  1969 non-null   int8 
 7   snap_CA       1969 non-null   int8 
 8   snap_TX       1969 non-null   int8 
 9   snap_WI       1969 non-null   int8 
dtypes: int64(3), int8(7)
memory usage: 59.7 KB


In [21]:
calendar.tail()

,wm_yr_wk,wday,month,year,d,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI
1964,11620,5,6,2016,1965,1,1,0,1,1
1965,11620,6,6,2016,1966,1,1,0,0,0
1966,11620,7,6,2016,1967,1,1,0,0,0
1967,11621,1,6,2016,1968,1,1,0,0,0
1968,11621,2,6,2016,1969,18,4,0,0,0


In [10]:
calendar = downtype(calendar)
train_evaluation = downtype(train_evaluation)
sales = downtype(sales)

In [23]:
train_evaluation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: category(6), int16(1299), int8(642)
memory usage: 95.8 MB


In [24]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype   
---  ------      -----   
 0   store_id    category
 1   item_id     category
 2   wm_yr_wk    int16   
 3   sell_price  float16 
dtypes: category(2), float16(1), int16(1)
memory usage: 45.8 MB


In [11]:
LAGS = [7,28]
WINDOWS = [7,28]
FIRST = 1942
LENGTH = 28

def demand_features(df):
    #Thêm hàng mới vào data
    """
        lag_ti thể hiện nhu cầu của sản phẩm ở ngày i trước đó
        Vd lag_t7 100 Hoobie1 tức là 7 ngày trước đó yêu cầu 100 sản phẩm hoobie1 để bán
        rolling_mean_ti_wj là giá trị trung bình động lag_ti trong j khoảng thời gian trước đó
        Vd rolling_mean_t7_w7 là giá trị trung bình động của lag_t7 trong 7 ngày trước(ngày đó là ngày thứ 7)
        rolling_mean sẽ làm mượt các giá trị nhiễu
    """
    for lag in LAGS:
        df[f'lag_t{lag}'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(lag))
        for w in WINDOWS:
            df[f'rolling_mean_t{lag}_w{w}'] = df.groupby(['id'])[f'lag_t{lag}'].transform(lambda x: x.rolling(w).mean().astype('float32'))
    return df

In [12]:
def prep_data(df, drop_d):
    # Loại bỏ drop_d ngày đầu tiên vì tính lag_t7 thì sẽ cần loại bỏ 7 cột đầu tiên k thì dữ liệu sẽ về NaN do ko có dữ liệu 7 ngày trước đó
    df = df.drop(["d_" + str(i+1) for i in range(drop_d)], axis=1)


    df = df.assign(id=df.id.str.replace("_evaluation", ""))

    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(FIRST + i) for i in range(LENGTH)])

    df = df.melt(id_vars=["id", "item_id", "store_id", "state_id", "dept_id", "cat_id"], var_name='d', value_name='demand')

    df = df.assign(d=df.d.str[2:].astype("int16"),
                   demand=df.demand.astype("float32"))
    # Thêm hàng lag_t và rolling_mean vào df
    df = demand_features(df)


    df = df[df.d > (drop_d + max(LAGS) + max(WINDOWS))]

    # Merge df với calendar dựa trên cột 'd'
    df = df.merge(calendar, how="left", on="d")
    # Merge với sales dựa trên các cột store_id, item_id, wm_yr_wk
    df = df.merge(sales, how="left", on=["store_id", "item_id", "wm_yr_wk"])

    df = df.drop(["wm_yr_wk"], axis=1)

    for v in ["item_id", "store_id", "state_id", "dept_id", "cat_id"]:
        df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]]).astype("int16") + 1


    x = list(set(df.columns) - {'id', 'd', 'demand'})

    test = df[df.d >= FIRST - max(LAGS) - max(WINDOWS) - LENGTH]
    df = df[df.d < FIRST]

    xtrain, xvalid, ytrain, yvalid = train_test_split(df[x], df["demand"], test_size=0.1, shuffle=True, random_state=19)

    train = lgb.Dataset(xtrain, label = ytrain)
    valid = lgb.Dataset(xvalid, label = yvalid)

    return train, valid, test, x

train, valid, test, x = prep_data(train_evaluation, 1000 - LENGTH)
test.head()

,id,item_id,store_id,state_id,dept_id,cat_id,d,demand,lag_t7,rolling_mean_t7_w7,...,rolling_mean_t28_w28,wday,month,year,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI,sell_price
25276210,HOBBIES_1_001_CA_1,1438,1,1,4,2,1858,0.0,4.0,1.142857,...,0.785714,3,2,2016,1,1,0,0,0,8.257812
25276211,HOBBIES_1_002_CA_1,1439,1,1,4,2,1858,0.0,0.0,0.285714,...,0.178571,3,2,2016,1,1,0,0,0,3.970703
25276212,HOBBIES_1_003_CA_1,1440,1,1,4,2,1858,0.0,0.0,0.428571,...,0.107143,3,2,2016,1,1,0,0,0,2.970703
25276213,HOBBIES_1_004_CA_1,1441,1,1,4,2,1858,0.0,2.0,1.857143,...,2.071429,3,2,2016,1,1,0,0,0,4.640625
25276214,HOBBIES_1_005_CA_1,1442,1,1,4,2,1858,1.0,0.0,0.857143,...,0.750000,3,2,2016,1,1,0,0,0,2.880859


In [27]:
print(x)

['rolling_mean_t28_w28', 'lag_t7', 'snap_CA', 'rolling_mean_t7_w7', 'item_id', 'event_name_1', 'snap_TX', 'lag_t28', 'wday', 'month', 'rolling_mean_t7_w28', 'rolling_mean_t28_w7', 'year', 'dept_id', 'cat_id', 'event_name_2', 'snap_WI', 'state_id', 'sell_price', 'store_id']


In [ ]:
train_df = pd.DataFrame(train.data)
train_df.columns = x
train_df.head()

In [13]:
def fit_model(train,valid):
    params = {
        'metric': 'rmse',
        'objective': 'poisson',
        'seed': 19,
        'force_row_wise': True,
        'learning_rate': 0.075,
        'lambda': 0.1,
        'num_leaves': 63,
        'colsample_bytree': 0.7,
        'bagging_freq': 1,
        'sub_row': 0.7
    }

    fit = lgb.train(params, train, num_boost_round=2000, valid_sets = [valid])
    return fit

In [ ]:
fit = fit_model(train, valid)

[LightGBM] [Info] Total Bins 1784
[LightGBM] [Info] Number of data points in the train set: 25053633, number of used features: 20
[LightGBM] [Info] Start training from score 0.205992


In [7]:
x_train = pd.read_csv("saved/data/train.csv")
x_valid = pd.read_csv("saved/data/valid.csv")
test = pd.read_csv("saved/data/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/saved/train.csv'

In [32]:
from sklearn.model_selection import StratifiedKFold